
## **Adopter Prediction Challenge**

 ~ Ankita, Ashok, Kaydee, Young
 
 ---

Website XYZ, a music-listening social networking website, follows the “freemium” business model. The website offers basic services for free, and provides a number of additional premium capabilities for a monthly subscription fee. We are interested in predicting which people would be likely to convert from free users to premium subscribers in the next 6 month period, if they are targeted by our promotional campaign.

### Dataset

We have a dataset from the previous marketing campaign which targeted a number of non-subscribers.

Features: 

```
1.   adopter (predictor class)
2.   user_id
3.   age
4.   male
5.   friend_cnt
6.   avg_friend_age
7.   avg_friend_male
8.   friend_country_cnt
9.   subscriber_friend_cnt
10.   songsListened
11.   lovedTracks
12.   posts
13.   playlists
14.   shouts
15.   good_country
16.   tenure
17.   *other delta variables*
```



### Task

The task is to build the best predictive model for the next marketing campaign, i.e., for predicting likely `adopters` (that is, which current non- subscribers are likely to respond to the marketing campaign and sign up for the premium service within 6 months after the campaign).

---

In [0]:
!pip3 install sklearn

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, classification_report, recall_score, f1_score, accuracy_score, precision_score

import xgboost as xgb
from sklearn.utils import shuffle

In [0]:
# setting fixed seed value for consistency in results
seed = 7
np.random.seed(seed)

In [0]:
# original dataset
data = pd.read_csv('https://drive.google.com/uc?export=view&id=1wctM0dYDj839zp6sTlFnDgCmFspXhDuW')

data.adopter.value_counts()

0    85142
1     1540
Name: adopter, dtype: int64

Checking to see if any features (especially adopter) needs to be encoded as int

In [0]:
# some housekeeping for metrics
recalls = {}
f1s = {}
precisions = {}
accuracies = {}

# splitting original dataset into features and predictor
X = data.iloc[:, data.columns != 'adopter']
y = data.iloc[:, data.columns == 'adopter']

# splitting the original dataset for cross-validation (0.7 train, 0.3 test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

print ("Number of train instances: {}".format(len(X_train)))
print ("Number of test instances: {}".format(len(X_test)))

Number of train instances: 60677
Number of test instances: 26005


## SMOTE splitting

We'll use SMOTE (Synthetic Minority Oversampling Technique) to create(synthesize) more samples of minority class. The recall score we got earlier might be less as we imputed more than 80% of the data to balance the dataset. 

Before we SMOTE the entire dataset, synthesizing around 58000 new instances of minority will not introduce enough variation in data for the models to learn. 

We decide that we will include only a subset of the majority class instances (4000) and synthsize 4000-1540=2460 new instances for minority class using SMOTE. That'll (hopefully) avoid our models from overfitting. 

In [0]:
# fetching the indices of minority instances
adopting_indices = np.array(data[data.adopter == 1].index)

# fetching indices of normal instances
non_adopting_indices = data[data.adopter == 0].index

# randomly select 1540 normal instances to create a partitioned balanced dataset
random_non_adopting_indices = np.random.choice(non_adopting_indices,
                                            5040,
                                            replace = False)
random_non_adopting_indices = np.array(random_non_adopting_indices)

# combining both the instance groups (minority and the new random set) 
undersampled_indices = np.concatenate([adopting_indices, random_non_adopting_indices])

# creating the undersampled dataset
undersampled_data = data.iloc[undersampled_indices, :]

# shuffling the new dataset
undersampled_data = shuffle(undersampled_data)

# storing the features(X) and predictor class(y)
X_undersample = undersampled_data.iloc[:, undersampled_data.columns != 'adopter']
y_undersample = undersampled_data.iloc[:, undersampled_data.columns == 'adopter']

print("Number of minority instances: {}\nNumber of normal instances: {} \nTotal: {}".format(len(undersampled_data[undersampled_data.adopter == 1]), 
                                                                                           len(undersampled_data[undersampled_data.adopter == 0]),
                                                                                           len(undersampled_data)))

Number of minority instances: 1540
Number of normal instances: 5040 
Total: 6580


In [0]:
# splitting original dataset into features and predictor
X = undersampled_data.iloc[:, data.columns != 'adopter']
y = undersampled_data.iloc[:, data.columns == 'adopter']

# splitting the original dataset for cross-validation (0.7 train, 0.3 test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

print ("Undersampled Data:")
print ("Number of train instances: {}".format(len(X_train)))
print ("Number of test instances: {}".format(len(X_test)))

Undersampled Data:
Number of train instances: 4606
Number of test instances: 1974


In [0]:
# sm = SMOTE(random_state = 12, ratio = None)
# X_train_smoted_np, y_train_smoted_np = sm.fit_sample(X_train, y_train)
# # X_train_smoted, y_train_smoted = sm.fit_sample(X_train, y_train.values.ravel())
# print(type(X_train_smoted_np))

In [0]:
# # checking the lengths of new training set

# print ("Number of SMOTEd instances: {}".format(len(X_train_smoted_np)))

# X_train.head()
# y_train_smoted_non_adopters = y_train_smoted_np[y_train_smoted_np == 1]
# y_train_smoted_adopters = y_train_smoted_np[y_train_smoted_np == 0]

# print ("Number of SMOTEd non-adopters (adopter = 0): {}".format(len(y_train_smoted_non_adopters)))
# print ("Number of SMOTEd adopters (adopter = 1): {}".format(len(y_train_smoted_adopters)))

We now have around 2792 instances each of both the classes, which is better than simple undersampling and having only 3080 instances in all.



For now we'll import smoted data from our R scripts since the above is taking time.

## XGBoost

In [0]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

num_rounds = 100

params = {
    'max_depth': 3,
    'n_estimators': 5000,
    'gamma': 0,
    'scale_pos_weight': 1,
    'eta': 0.1,
    'reg_alpha': 0.05,
    'objective': 'binary:logistic',
    'seed': 7
}

test_train_split = [(dtest, 'test'), (dtrain, 'train')]

boost = xgb.train(params,
                 dtrain,
                 num_rounds, 
                 test_train_split)

[0]	test-error:0.212259	train-error:0.22145
[1]	test-error:0.211753	train-error:0.214286
[2]	test-error:0.211753	train-error:0.213851
[3]	test-error:0.215299	train-error:0.215371
[4]	test-error:0.222898	train-error:0.211246
[5]	test-error:0.223404	train-error:0.209944
[6]	test-error:0.223404	train-error:0.209075
[7]	test-error:0.222391	train-error:0.209292
[8]	test-error:0.219858	train-error:0.209075
[9]	test-error:0.220365	train-error:0.208858
[10]	test-error:0.214286	train-error:0.208207
[11]	test-error:0.212766	train-error:0.209726
[12]	test-error:0.215299	train-error:0.210595
[13]	test-error:0.212259	train-error:0.210595
[14]	test-error:0.212766	train-error:0.209075
[15]	test-error:0.214792	train-error:0.209509
[16]	test-error:0.212259	train-error:0.208858
[17]	test-error:0.21074	train-error:0.207338
[18]	test-error:0.211753	train-error:0.207338
[19]	test-error:0.210233	train-error:0.206904
[20]	test-error:0.20922	train-error:0.206253
[21]	test-error:0.2077	train-error:0.205819
[22

In [0]:
y_pred = boost.predict(dtest)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0

# acc_val = accuracy_score(y_pred, y_test)
# f1_val = f1_score(y_pred, y_test)
# recall_val = recall_score(y_pred, y_test)
# prec_val = precision_score(y_pred, y_test)

print (accuracy_score(y_pred, y_test))
print (f1_score(y_pred, y_test))
print (recall_score(y_pred, y_test))
print (precision_score(y_pred, y_test))

recalls.update({len(undersampled_data[undersampled_data.adopter == 0]) : recall_score(y_pred, y_test)})
f1s.update({len(undersampled_data[undersampled_data.adopter == 0]) : f1_score(y_pred, y_test)})
precisions.update({len(undersampled_data[undersampled_data.adopter == 0]) : precision_score(y_pred, y_test)})
accuracies.update({len(undersampled_data[undersampled_data.adopter == 0]) : accuracy_score(y_pred, y_test)})

print(recalls)
print(f1s)
print(precisions)
print(accuracies)

0.7948328267477204
0.4222539229671898
0.5714285714285714
0.334841628959276
{5040: 0.5714285714285714}
{5040: 0.4222539229671898}
{5040: 0.334841628959276}
{5040: 0.7948328267477204}


Let's try standardising the data

In [0]:
# from sklearn.preprocessing import StandardScaler

# scaler = preprocessing.StandardScaler()
# scaler.fit(X_train.values) 

In [0]:
# dtrain = xgb.DMatrix(X_train, y_train)
# dtest = xgb.DMatrix(X_test, y_test)

# num_rounds = 50

# params = {
#     'max_depth': 3,
#     'eta': 0.1,
#     'objective': 'binary:logistic',
#     'seed': 7
# }

# test_train_split = [(dtest, 'test'), (dtrain, 'train')]

# boost = xgb.train(params,
#                  dtrain,
#                  num_rounds, 
#                  test_train_split)

No effect seen after standardising

In [0]:
# y_pred = boost.predict(dtest)

# y_pred[y_pred > 0.5] = 1
# y_pred[y_pred <= 0.5] = 0

# type(y_pred)

In [0]:
# # predictions on unlabelled set
unseen_data = pd.read_csv('https://drive.google.com/uc?export=view&id=1yVPwqGQC2gkhF2bcbue9j3184ryAJRtG')
unseen_data = xgb.DMatrix(unseen_data)

y_pred = boost.predict(unseen_data)
# y_pred[y_pred > 0.5] = 1
# y_pred[y_pred <= 0.5] = 0

y_pred = pd.DataFrame({'Adopters': y_pred })

In [0]:
# # testing the model on provided test dataset
np.savetxt("predictions.csv", y_pred , delimiter=",")
from google.colab import files
files.download('predictions.csv')